In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout # type: ignore
from keras.optimizers import Adam
import cv2 as cv
from PIL import Image

In [2]:
# Load the data
split_df = pd.read_csv('split_df.csv', encoding='utf-8-sig')


In [3]:
split_df

,Table 1,Spectacles,Age,Gender,Name,id,Eye,VA,CDR,Diagnosis,IOP,Image
0,زرق أيمن تم معالجته جراحيا,0.0,64,0,شاهين ميرو,1.0,0,0.6,0.5,0,14.0,Data/SHAHEN_MERO Optic nerve.png_ left.png
1,زرق أيمن تم معالجته جراحيا,0.0,64,0,شاهين ميرو,1.0,1,0.7,0.9,1,23.0,Data/SHAHEN_MERO Optic nerve.png_ left.png
2,زرق متقدم,1.0,62,0,أيمن سمارة,2.0,0,NaN,NaN,1,25.0,Data/SHAHEN_MERO Optic nerve.png_ right.png
3,زرق متقدم,1.0,62,0,أيمن سمارة,2.0,1,9.0,0.8,1,40.0,Data/SHAHEN_MERO Optic nerve.png_ right.png
4,لا يوجد زرق,0.0,48,1,رندة ناصر,3.0,0,1.0,0.5,0,15.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
85,زرق أيسر تم معالجته جراحيا,1.0,55,1,زريفة المحمد,43.0,1,NaN,NaN,0,NaN,Data/BASEL_BASH Optic nerve.png_ left.png
86,مريض زرقي ثانوي بعد استخراج ساد لا يوجد اصابة ...,0.0,71,0,هيثم قزاز,44.0,0,0.3,0.5,1,15.0,Data/BASEL_BASH Optic nerve.png_ right.png
87,مريض زرقي ثانوي بعد استخراج ساد لا يوجد اصابة ...,0.0,71,0,هيثم قزاز,44.0,1,0.6,0.7,1,16.0,Data/BASEL_BASH Optic nerve.png_ right.png
88,لا يوجد زرق,1.0,65,0,حمود غشام,45.0,0,0.5,0.3,0,15.0,Data/NEZAR_HELOU Optic nerve.png_ left.png


In [4]:
split_df['Age'].value_counts()

Age
64    10
65     6
25     4
67     4
73     4
62     4
55     4
58     4
70     4
50     2
59     2
29     2
17     2
48     2
37     2
52     2
32     2
45     2
60     2
44     2
69     2
43     2
12     2
66     2
24     2
53     2
56     2
39     2
68     2
61     2
76     2
71     2
Name: count, dtype: int64

In [5]:
split_df = split_df.drop(['Table 1', 'Gender', 'id', 'Age', 'Spectacles'], axis=1)

In [6]:
split_df['Image'].isnull().sum()

4

In [7]:
# split_df['Image']
split_df[split_df['Image'].isnull()]

,Name,Eye,VA,CDR,Diagnosis,IOP,Image
4,رندة ناصر,0,1.0,0.5,0,15.0,NaN
5,رندة ناصر,1,1.0,0.4,0,14.0,NaN
18,خديجة كبة,0,0.5,0.3,1,36.0,NaN
19,خديجة كبة,1,0.7,0.3,0,17.0,NaN


In [8]:
split_df = split_df.drop(['Name','Eye'], axis=1)

In [9]:
split_df.dropna(inplace=True)

In [10]:
split_df

,VA,CDR,Diagnosis,IOP,Image
0,0.6,0.5,0,14.0,Data/SHAHEN_MERO Optic nerve.png_ left.png
1,0.7,0.9,1,23.0,Data/SHAHEN_MERO Optic nerve.png_ left.png
3,9.0,0.8,1,40.0,Data/SHAHEN_MERO Optic nerve.png_ right.png
6,0.2,0.7,1,10.0,Data/AYMAN_SAMARA_20240121_124719_Disc_3D_R_SI...
7,0.1,0.7,1,14.0,Data/AYMAN_SAMARA_20240121_124719_Disc_3D_R_SI...
...,...,...,...,...,...
84,0.2,0.8,1,10.0,Data/BASEL_BASH Optic nerve.png_ left.png
86,0.3,0.5,1,15.0,Data/BASEL_BASH Optic nerve.png_ right.png
87,0.6,0.7,1,16.0,Data/BASEL_BASH Optic nerve.png_ right.png
88,0.5,0.3,0,15.0,Data/NEZAR_HELOU Optic nerve.png_ left.png


In [11]:
orb = cv.ORB_create()

def extract_orb_features(image_path):
    image = cv.imread(image_path)
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    keypoints, descriptors = orb.detectAndCompute(gray, None)
    return descriptors.flatten() if descriptors is not None else np.zeros(500)

In [12]:
split_df['ORB_features'] = split_df['Image'].apply(extract_orb_features)

In [13]:
split_df.count() 

VA              66
CDR             66
Diagnosis       66
IOP             66
Image           66
ORB_features    66
dtype: int64

In [14]:
# Load and preprocess images for each patient
image_paths =  split_df['Image'].tolist()


In [15]:
image_paths

['Data/SHAHEN_MERO Optic nerve.png_ left.png',
 'Data/SHAHEN_MERO Optic nerve.png_ left.png',
 'Data/SHAHEN_MERO Optic nerve.png_ right.png',
 'Data/AYMAN_SAMARA_20240121_124719_Disc_3D_R_SINGLE_6mm_512x112.png',
 'Data/AYMAN_SAMARA_20240121_124719_Disc_3D_R_SINGLE_6mm_512x112.png',
 'Data/RANDA_NASER_2 Optic nerve.png_ right.png',
 'Data/RANDA_NASER_2 Optic nerve.png_ right.png',
 'Data/MOUSTAFA_MALA_ALI Optic nerve.png_ left.png',
 'Data/MOUSTAFA_MALA_ALI Optic nerve.png_ left.png',
 'Data/MOUSTAFA_MALA_ALI Optic nerve.png_ right.png',
 'Data/MOUSTAFA_MALA_ALI Optic nerve.png_ right.png',
 'Data/6361_NAHLA_ABO_QORA_left.png',
 'Data/6361_NAHLA_ABO_QORA_left.png',
 'Data/LATEFA_IBRAHEM Optic nerve.png_ left.png',
 'Data/LATEFA_IBRAHEM Optic nerve.png_ left.png',
 'Data/FATEMA_FARHAN Optic nerve.png_ left.png',
 'Data/FATEMA_FARHAN Optic nerve.png_ left.png',
 'Data/FATEMA_FARHAN Optic nerve.png_ right.png',
 'Data/FATEMA_FARHAN Optic nerve.png_ right.png',
 'Data/SOBHI_WAZAN Optic ner

In [16]:
# images = [np.array(Image.open(path).convert('RGB').resize((200, 200))) for path in image_paths]  # Load and resize images
# X_images = np.array(images) / 255.0  # Normalize pixel values to range [0, 1]

In [17]:
# Drop columns not used for prediction
y_data = split_df['Diagnosis']
X_data = split_df.drop(['Image','Diagnosis'], axis=1)

In [18]:
# # image_paths[0]
# img = cv.imread('Data/SHAHEN_MERO Optic nerve.png_ left.png')
# cv.imshow('Image', img)

# cv.waitKey(0)
# cv.destroyAllWindows()

In [19]:
# X_images = []
# # cv.imshow('',X_images)
# # cv.waitKey(0)
# # cv.destroyAllWindows()
# for i in range(len(image_paths)):
#     img = cv.imread(image_paths[i])
#     flattened_image = img.flatten()
#     X_images.append(flattened_image)

In [20]:
# X_images = pd.DataFrame(X_images,columns='images')
# X_images = pd.DataFrame({'Image': X_images})


In [21]:
# X_images

In [22]:
# X_images.isnull().sum()

In [23]:

X_data

,VA,CDR,IOP,ORB_features
0,0.6,0.5,14.0,"[13, 254, 157, 23, 145, 206, 39, 184, 242, 223..."
1,0.7,0.9,23.0,"[13, 254, 157, 23, 145, 206, 39, 184, 242, 223..."
3,9.0,0.8,40.0,"[218, 169, 144, 178, 155, 153, 16, 206, 171, 5..."
6,0.2,0.7,10.0,"[198, 98, 254, 209, 238, 107, 251, 200, 255, 1..."
7,0.1,0.7,14.0,"[198, 98, 254, 209, 238, 107, 251, 200, 255, 1..."
...,...,...,...,...
84,0.2,0.8,10.0,"[175, 174, 204, 61, 232, 27, 108, 255, 23, 242..."
86,0.3,0.5,15.0,"[133, 254, 143, 146, 236, 244, 167, 222, 142, ..."
87,0.6,0.7,16.0,"[133, 254, 143, 146, 236, 244, 167, 222, 142, ..."
88,0.5,0.3,15.0,"[96, 42, 150, 152, 73, 233, 160, 72, 174, 189,..."


In [24]:
X_data.isnull().sum()

VA              0
CDR             0
IOP             0
ORB_features    0
dtype: int64

In [25]:
# X_train = np.concatenate((X_data, X_images), axis=1)
# X_data['images'] = X_images['Image']
# for i in range(len(X_images)):
#     X_data.at[i, 'images'] = X_images.at[i, 'Image']


In [26]:
X_data.count()

VA              66
CDR             66
IOP             66
ORB_features    66
dtype: int64

In [27]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.1, random_state=44, shuffle=True)


In [28]:
# Define CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    MaxPooling2D((2, 2)),
    Dropout(rate=0.2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid') 
])

In [29]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 99, 99, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 99, 99, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 48, 48, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 128)       7

In [31]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [ ]:
test_loss, test_acc = model.evaluate(X_images_test, y_test)
print(f"Test Accuracy: {test_acc}")

1/1 [==============================] - 0s 111ms/step - loss: 0.9338 - accuracy: 0.5714
Test Accuracy: 0.5714285969734192


In [ ]:
# Evaluate the model
train_score = model.evaluate(X_images_train, y_train)
test_score = model.evaluate(X_images_test, y_test)
print('CNN Train Score:', train_score)
print('CNN Test Score:', test_score)


1/1 [==============================] - 0s 119ms/step - loss: 0.9338 - accuracy: 0.5714
CNN Train Score: [0.34277939796447754, 0.8245614171028137]
CNN Test Score: [0.9337977170944214, 0.5714285969734192]


In [ ]:
# Make predictions on the test set
predictions = model.predict(X_images_test)


1/1 [==============================] - 0s 284ms/step


In [ ]:
# # Generate classification report and confusion matrix
# print('Classification Report:')
# print(classification_report(y_test, predictions))

# print('Confusion Matrix:')
# print(confusion_matrix(y_test, predictions))


In [ ]:
from keras.backend import clear_session
clear_session()